1. 문서의 내용을 읽는다 
2. 문서를 쪼갠다 
   - 토큰 수 초과로 답변을 생성하지 못할 수 있고 
   - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림 
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

# 1. 문서의 내용을 읽는다

In [1]:
#document loader x 

%pip install python-docx

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 32.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


python-docx
- paragraph (단락)

- 한 문장씩 full_text에 넣기 

In [12]:
from docx import Document

document = Document('./tax.docx')
print(f'document == {dir(document)}')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'
    
    
    

document == ['_Document__body', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_block_width', '_body', '_element', '_parent', '_part', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'core_properties', 'element', 'inline_shapes', 'iter_inner_content', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']
paragraph == 소득세법
paragraph == [시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]
paragraph == 기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
paragraph == 기획재정부(소득세제과(근로소득)) 044-215-4216
paragraph == 기획재정부(재산세제과(양도소득세)) 044-215-4314
paragraph == 기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236
paragraph == 
paragraph == 제1장 총칙 <개정 2009. 12. 31.>	
pa

In [13]:
full_text

'소득세법\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(재산세제과(양도소득세)) 044-215-4314\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236\n\n제1장 총칙 <개정 2009. 12. 31.>\t\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n[본조신설 2009. 12. 31.]\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자\n3. 내국법인\n4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 이하

# 2. 문서를 쪼갠다

In [14]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [19]:
import tiktoken 

encoder = tiktoken.encoding_for_model("gpt-4o")
encoding = encoder.encode(full_text) #텍스트를 숫자로
decoded = encoder.decode(encoding) #숫자를 텍스트로

In [21]:
print(encoding) # 토큰수 

[11226, 64328, 11734, 22070, 198, 58, 5637, 15719, 220, 1323, 19, 13, 220, 16, 13, 220, 16, 54569, 723, 22070, 73888, 14377, 3204, 2546, 16684, 11, 220, 1323, 18, 13, 220, 899, 13, 220, 2911, 4213, 124313, 13915, 7329, 1592, 4283, 60576, 16012, 180899, 7, 11226, 64328, 11734, 8021, 8562, 7, 131266, 11226, 64328, 11, 168944, 11226, 64328, 915, 220, 35132, 12, 21625, 12, 34096, 22, 198, 4283, 60576, 16012, 180899, 7, 11226, 64328, 11734, 8021, 8562, 7, 32481, 3710, 11226, 64328, 915, 220, 35132, 12, 21625, 12, 34096, 21, 198, 4283, 60576, 16012, 180899, 7, 16012, 17198, 11734, 8021, 8562, 7, 21902, 5827, 11226, 64328, 11734, 915, 220, 35132, 12, 21625, 12, 36948, 19, 198, 4283, 60576, 16012, 180899, 7, 16668, 110223, 11734, 8021, 8562, 7, 2186, 5947, 11226, 64328, 11, 33628, 19388, 11226, 64328, 915, 220, 35132, 12, 21625, 12, 36645, 21, 279, 8021, 16, 7681, 64009, 90720, 464, 13915, 7329, 220, 1179, 24, 13, 220, 899, 13, 220, 2911, 13, 29, 31711, 8021, 16, 14844, 7, 25816, 9871, 8, 6548

In [20]:
decoded

'소득세법\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(재산세제과(양도소득세)) 044-215-4314\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236\n\n제1장 총칙 <개정 2009. 12. 31.>\t\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n[본조신설 2009. 12. 31.]\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자\n3. 내국법인\n4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 이하

In [18]:
len(encoding)

186506

In [22]:
import tiktoken 

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    encoding = encoder.encode(full_text) #숫자형식
    total_token_count = len(encoding) #186506
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)#청크로 나눈후 문자로 변경 
        text_list.append(decoded)
    
    return text_list

In [23]:
chunk_list = split_text(full_text, 1500)

In [24]:
chunk_list

['소득세법\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n기획재정부(소득세제과(근로소득)) 044-215-4216\n기획재정부(재산세제과(양도소득세)) 044-215-4314\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236\n\n제1장 총칙 <개정 2009. 12. 31.>\t\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n[본조신설 2009. 12. 31.]\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n[본조신설 2009. 12. 31.]\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.\n1. 거주자\n2. 비거주자\n3. 내국법인\n4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 이

# 3. 문서 임베딩

In [25]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [26]:
import chromadb

chroma_client = chromadb.Client()

In [28]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

UniqueConstraintError: Collection tax_collection already exists

In [31]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os 

openai_api_key = os.getenv('openai_api_key')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [32]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)


In [33]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [34]:
len(id_list)

125

In [35]:
len(chunk_list)

125

In [36]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 유사도 검색

In [41]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [42]:
retrieved_doc

{'ids': [['5', '31', '7']],
 'distances': [[1.141217827796936, 1.1508026123046875, 1.163649082183838]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품\n나. 「국가보안법」에 따라 받는 상금과 보로금\n다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상\n라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.\n마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품\n바. 「문화재보호법」에 따라 국가지정문화재로 지정된 서화ㆍ골동품의 양도로 발생하는 소득\n사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득\n아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득\n\u3000\u3000\u3000\u30001) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금\n\u3000\u3000\u3000\u30002) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대\n\u3000\u3000\u3000\u30003) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액\n\u3000\

In [43]:
retrieved_doc['documents'][0]

['우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품\n나. 「국가보안법」에 따라 받는 상금과 보로금\n다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상\n라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.\n마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품\n바. 「문화재보호법」에 따라 국가지정문화재로 지정된 서화ㆍ골동품의 양도로 발생하는 소득\n사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득\n아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득\n\u3000\u3000\u3000\u30001) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금\n\u3000\u3000\u3000\u30002) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대\n\u3000\u3000\u3000\u30003) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액\n\u3000\u3000\u3000\u30004) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액\n\u3000\u3000\u3000\u30005) 종교관련종사자가 기획재정부령으로 정하는 사택을 제공받아 

# 5. llm 질의

In [44]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [45]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
    {"role": "user", "content": query}
  ]
)

In [46]:
response

ChatCompletion(id='chatcmpl-A2grdbmAm7BFDcwPCDT7vgkkConAJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='연봉 5천만원인 직장인의 소득세 계산 방법에 대해 설명드리겠습니다. 소득세는 근로소득공제와 기본공제 등을 적용한 후 과세표준에 따라 세율을 적용합니다. 다음은 소득세 계산 절차입니다:\n\n1. **근로소득공제 계산**:\n    - 연봉 5천만원에 대한 근로소득공제를 계산합니다.\n    - 연봉 3천만원 초과 5천만원 이하의 경우: 1,200만원 + (연봉 - 3천만원) × 15%\n      \n      따라서, 근로소득공제는:\n      \\[\n      1,200만원 + (5,000만원 - 3,000만원) \\times 0.15 = 1,200만원 + 300만원 = 1,500만원\n      \\]\n\n2. **과세표준 계산**:\n    - 근로소득금액 = 연봉 - 근로소득공제\n    - 과세표준 = 근로소득금액 - 기본공제(1인당 150만원)\n\n    따라서, 과세표준은:\n    \\[\n    5,000만원 - 1,500만원 - 150만원 = 3,350만원\n    \\]\n\n3. **소득세 계산**:\n    - 과세표준 3,350만원에 대해 누진세율을 적용합니다.\n    - 1,200만원 이하: 6%\n    - 1,200만원 초과 4,600만원 이하: 15%-누진공제액 108만원\n\n    따라서, 소득세는:\n    \\[\n    3,350만원 \\times 0.15 - 108만원 = 502.5만원 - 108만원 = 394.5만원\n    \\]\n\n4. **근로소득세액공제 적용**:\n    - 총급여액 3,300만원 초과 7천만원 이하의 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]\n    \n    따라

In [48]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세 계산 방법에 대해 설명드리겠습니다. 소득세는 근로소득공제와 기본공제 등을 적용한 후 과세표준에 따라 세율을 적용합니다. 다음은 소득세 계산 절차입니다:\n\n1. **근로소득공제 계산**:\n    - 연봉 5천만원에 대한 근로소득공제를 계산합니다.\n    - 연봉 3천만원 초과 5천만원 이하의 경우: 1,200만원 + (연봉 - 3천만원) × 15%\n      \n      따라서, 근로소득공제는:\n      \\[\n      1,200만원 + (5,000만원 - 3,000만원) \\times 0.15 = 1,200만원 + 300만원 = 1,500만원\n      \\]\n\n2. **과세표준 계산**:\n    - 근로소득금액 = 연봉 - 근로소득공제\n    - 과세표준 = 근로소득금액 - 기본공제(1인당 150만원)\n\n    따라서, 과세표준은:\n    \\[\n    5,000만원 - 1,500만원 - 150만원 = 3,350만원\n    \\]\n\n3. **소득세 계산**:\n    - 과세표준 3,350만원에 대해 누진세율을 적용합니다.\n    - 1,200만원 이하: 6%\n    - 1,200만원 초과 4,600만원 이하: 15%-누진공제액 108만원\n\n    따라서, 소득세는:\n    \\[\n    3,350만원 \\times 0.15 - 108만원 = 502.5만원 - 108만원 = 394.5만원\n    \\]\n\n4. **근로소득세액공제 적용**:\n    - 총급여액 3,300만원 초과 7천만원 이하의 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]\n    \n    따라서, 근로소득세액공제는:\n    \\[\n    74만원 - [(5,000만원 - 3,300만원) \\times 0.008] = 74만원 - 13.6만원 = 60.4만원\n    \\]\n    \n    최종 소득세:\n    \\[\n    394.5만원 - 60.4만원 = 334